In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Check TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")

# 1. LOAD DATA
# We use Fashion MNIST: 60,000 grayscale images (28x28 pixels) of 10 clothing categories.
# This simulates collecting images for our Edge AI model.
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Define class names for readability later
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print("Data loaded successfully.")
print(f"Training images shape: {train_images.shape}")

TensorFlow Version: 2.19.0
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Data loaded successfully.
Training images shape: (60000, 28, 28)


In [ ]:
# 2. PREPROCESSING
# Normalize pixel values to be between 0 and 1.
# This helps the model learn faster and more accurately.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape images to include the "channel" dimension.
# Most computer vision models expect shape: (Batch_Size, Height, Width, Channels)
# Since these are grayscale, Channels = 1.
train_images = train_images.reshape((-1, 28, 28, 1))
test_images = test_images.reshape((-1, 28, 28, 1))

print("Data preprocessed: Normalized and Reshaped.")

Data preprocessed: Normalized and Reshaped.


In [ ]:
# 3. BUILD & TRAIN MODEL
# We build a lightweight model suitable for Edge devices (fewer layers = faster).
model = tf.keras.Sequential([
    # The "Eyes": Scans for features (32 filters, 3x3 size)
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),

    # The "Compressor": Reduces size to make processing faster
    tf.keras.layers.MaxPooling2D((2, 2)),

    # The "Connector": Flattens 2D picture into 1D list of numbers
    tf.keras.layers.Flatten(),

    # The "Brain": Thinks about what the features mean
    tf.keras.layers.Dense(128, activation='relu'),

    # The "Output": Gives a score for each of the 10 categories
    tf.keras.layers.Dense(10)
])

# compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model (We use only 3 epochs to keep it quick for this assignment)
print("\nStarting training...")
model.fit(train_images, train_labels, epochs=3, validation_data=(test_images, test_labels))
print("Training complete.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Starting training...
Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.8135 - loss: 0.5285 - val_accuracy: 0.8926 - val_loss: 0.3053
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.9019 - loss: 0.2680 - val_accuracy: 0.9012 - val_loss: 0.2751
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9209 - loss: 0.2134 - val_accuracy: 0.9075 - val_loss: 0.2514
Training complete.


In [ ]:
# 4. CONVERT TO TFLITE
# Edge devices (like Raspberry Pi) need smaller, faster models.
# We convert the standard model to TensorFlow Lite format.

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# OPTIMIZATION (Quantization)
# This makes the model 4x smaller by simplifying the numbers
# (e.g., changing high-precision decimals to integers) without losing much accuracy.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the converted model to a file
with open('edge_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted and saved as 'edge_model.tflite'")

Saved artifact at '/tmp/tmpzd2gcgqt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134389572365648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134389572366800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134389572366032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134389572367184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134389572366992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134389572367568: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted and saved as 'edge_model.tflite'


In [ ]:
# 5. RUN INFERENCE (SIMULATION)
# This simulates the "Edge" device making a decision using the Lite model.

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="edge_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Pick a random image from the test set to test
# We use image #0 (which is usually an Ankle Boot)
input_data = test_images[0:1].astype(np.float32)
input_index = input_details[0]['index']

# Feed the image into the model
interpreter.set_tensor(input_index, input_data)

# Run the prediction!
interpreter.invoke()

# Get the result
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data)

print("\n--- Edge AI Prediction Result ---")
print(f"Predicted Class: {class_names[predicted_class]}")
print(f"Actual Label:    {class_names[test_labels[0]]}")

if predicted_class == test_labels[0]:
    print("Result: SUCCESS! The Edge model predicted correctly.")
else:
    print("Result: Incorrect prediction.")


--- Edge AI Prediction Result ---
Predicted Class: Ankle boot
Actual Label:    Ankle boot
Result: SUCCESS! The Edge model predicted correctly.


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
